# Introduction to vector and raster analysis

This notebook introduces concepts, tools and data processing necassarry to do quantative analysis with both raster and vector data. We will cover:

- Vector data - buildings (polygons), streets (linestrings).
- Raster data - land use
- Combining vectors and rasters
- Generating new data
- Two ways of using Spatial Relationships
- Machine Learning
- Some cloud-native data formats

The focus will be on quantatative urban planning, specifically urban morphometrics, which we define as:

> A study of urban form through the means of quantitative assessment of its constituent elements - streets, buildings and their configurations.

We focus on a simplified version of the work carried out in the [PRIMUS project](https://uscuni.org/himoc/). The project aims to deliniate areas with similar morphological character - with similar configurations of streets, buildings and the space between them. The similarity reflect the land use use, planning paradigms and conditions under which they were constructed.  The project covers every street and building in 6 contiguous European countries, ~120 million data points with 60 features each, so the tools and concepts scale.

The principles and tools are used for completely different applications - ice velocity in Antarctica, population estimation, suggesting store locations, mobility analysis, insurance and others.

The buildings and street data comes from Overturemaps, which is a superset of Openstreetmap in which other data is included. The land use  dataset used comes from the Corine Land Cover, specifically the uk subset. All the tools used in the analysis of this data are open, demonstrating what can be done with open-source software. 

For a more detailed look into this type work visit the ``momepy`` documentaiton: https://docs.momepy.org/.
Or have a look at the full PRIMUS project: https://uscuni.org/himoc/ and related slides: https://uscuni.org/talks/slides/202506_ISUF.html#/title-slide .

In [ ]:
# import the requried libraries
import geopandas as gpd
import libpysal
import matplotlib.pyplot as plt
import numpy as np
import momepy
import shapely
import neatnet
from overturemaps import record_batch_reader

We will focus the analysis on Norwich.

In [ ]:
place = "Norwich, UK"
epsg_uk = 27700

In [ ]:
# norwich_centre = gpd.tools.geocode(place)

norwich_centre = gpd.GeoSeries(
    [shapely.Point(1.29626, 52.62858)], name="geometry", crs="epsg:4326"
)

In [ ]:
norwich_radius = norwich_centre.to_crs(epsg=epsg_uk).buffer(2_000)

## Buildings

One of the core elements of the analysis are detailed building footprints. To get the data, we first read all the possible information for buildings from Overture maps for our specified location. Since  Overture data is a combination of Openstreetmap buioldings, 3rd-party buildings and satelite derived buildings footprints we will need to carry out some preprocessing to adress data inconsistencies and quality problems.

We are reading a small part of a large file with billions of buildings and streets and other geospatial objects. The file format is `geoparquet` and does not require any additional services to support partial reading of data - no servers, no code. You only need the file and to upload it in a compatible storage engine like S3 or Azure Blob Storage. You can write such files with ```gdf.to_parquet(...)```


In [ ]:


# reader = record_batch_reader(
#     "building", bbox=tuple(norwich_radius.to_crs(epsg=4326).bounds.values[0])
# )
# res = reader.read_all()
# buildings = gpd.GeoDataFrame.from_arrow(res)
# buildings = buildings.set_crs(epsg=4326)

In [ ]:
# buildings.to_parquet('~/space_east/data/buildings.parquet')
buildings = gpd.read_parquet('~/space_east/data/buildings.parquet')

Next, we filter out unwated structures.

In [ ]:

unwanted = ['garage',
 'garages',
 'shed',
 'bungalow',
 'roof',
 'parking',
 'kiosk',
 'hut',
 'pavilion',
 'greenhouse',
 'cabin',
 'toilets',
 'farm_auxiliary',
 'guardhouse']

In [ ]:
buildings = buildings[~buildings['class'].isin(unwanted)]

Next, we filter out polygons that are not single structures.

In [ ]:
buildings = buildings[buildings.geom_type == "Polygon"]

Lastly, we drop structures that have an area of less than $25m^2$. Before we do this, we have to reproject the data. 

In [ ]:
buildings = buildings[["geometry"]].to_crs(epsg=27700)
buildings = buildings[buildings.area > 25].reset_index(drop=True)
buildings.head()

### Streets

Similarly, the streets come from Overture data and are also a subset of OpenStreetMap.

In [1]:
# reader = record_batch_reader(
#     "segment", bbox=tuple(norwich_radius.to_crs(epsg=4326).bounds.values[0])
# )
# res = reader.read_all()
# streets = gpd.GeoDataFrame.from_arrow(res)
# streets = streets.set_crs(epsg=4326)
# streets

In [ ]:
# streets.to_parquet('~/space_east/data/streets.parquet')
streets = gpd.read_parquet('~/space_east/data/streets.parquet')

We carry out some preprocessing, similar to the buildings to drop unwated street segments such as tunnels, underground segments, or footpaths.

In [ ]:
## service road removed
approved_roads = [
    "living_street",
    "motorway",
    "motorway_link",
    "pedestrian",
    "primary",
    "primary_link",
    "residential",
    "secondary",
    "secondary_link",
    "tertiary",
    "tertiary_link",
    "trunk",
    "trunk_link",
    "unclassified",
]
streets = streets[streets["class"].isin(approved_roads)]

In [ ]:
def to_drop_tunnel(row):
    """Find whether or not a road segment has a tunnel thats more than 50 metres."""
    tunnel_length = row.geometry.length
    flags = row.road_flags

    total_tunnel_proportion = -1
    for flag in flags:
        if "values" in flag and ("is_tunnel" in flag["values"]):
            # between could be missing to show the whole thing is a tunnel
            total_tunnel_proportion = (
                0.0 if total_tunnel_proportion < 0 else total_tunnel_proportion
            )
            # betweencould be None to indicate the whole thing is a tunnel
            if ("between" in flag) and (flag["between"] is not None):
                s, e = flag["between"][0], flag["between"][1]
                total_tunnel_proportion += e - s

    return (total_tunnel_proportion * tunnel_length) > 50 or (total_tunnel_proportion == 0.0)

## drop tunnels
to_filter = streets.loc[~streets.road_flags.isna(),].to_crs(epsg=epsg_uk)
tunnels_to_drop = to_filter.apply(to_drop_tunnel, axis=1)
streets = streets.drop(to_filter[tunnels_to_drop].index)

The downloaded streets reflect the routing structure of the network, however we are more interested in its overall physical representation. For example dual-carriageways should be repreesented as one line, and round-abouts as an intersection of roads. We generate this representation from the routing represetntation using the open source Python package `neatnet`.

In [ ]:
# streets.explore()

In [ ]:
streets = streets.to_crs(epsg=epsg_uk)
streets = streets.sort_values("id")[["id", "geometry", "class"]].reset_index(
    drop=True
)

## simplify
simplified = neatnet.neatify(
    streets,
    exclusion_mask=buildings.geometry,
    artifact_threshold_fallback=7,
)



In [ ]:
streets = simplified.copy()

In [ ]:
# streets.explore()

## Issues with the OpenStreetmap data & Corine Land Cover. Combining Vectors and Rasters

Very high quality data is required to do quantatative urban planning.In some places Openstreetmap/Overture has issues: 

1. Misclassified or missing tags. For example, some sheds are not defined as such. Some roofs and power lines are identified as buildings (sometimes this happens in cadastres).
2. Deliniation discrepencies. For example, row houses get merged/split sometimes with reason, othertimes not. But in many cases the polyons do not reflect conssistently the physical character of the building.

To adress these issues somewhat we will use [Corine land cover data](https://land.copernicus.eu/en/products/corine-land-cover). Corine land cover is  `a pan-European land cover and land use inventory with 44 thematic classes, ranging from broad forested areas to individual vineyards` . However, its classification in urban areas is limited to only two classes - continuous urban areas and discontinuous urban areas. 

We limit the analysis to these types of areas by dropping all buildings outside of them. This will help adress the first problem. 

In [ ]:
import xvec
import rioxarray

In [ ]:
# read only the region part of the raster data
corine = rioxarray.open_rasterio('./data/U2018_CLC2018_V2020_20u1.tif')

In [ ]:
corine

In [ ]:
# setup geometry list to burn into the raster
building_utm = buildings.copy()
building_utm['num_index'] = np.arange(1, building_utm.shape[0] + 1)
building_utm['geometry'] = building_utm.representative_point()
building_utm = building_utm.to_crs(corine.rio.crs)

# extract building classification from corine raster
aggregated_iterative = corine.xvec.extract_points(
    building_utm.geometry,
    x_coords="x",
    y_coords="y",
)

# append to the building dataframe and store in the  results array
res = aggregated_iterative.to_pandas()
res = gpd.GeoDataFrame(res.T.reset_index(), crs=building_utm.crs)
res.columns = ['geometry', 'corine']
res.name = ''

In [ ]:
# subset data
# 1 - Urban Continuous
# 2 - Urban Discontinuous
buildings = buildings[res.corine < 3].reset_index(drop=True)

## Generated data

### Tessellation

We can generate a spatial unit using morphological tessellation, which aims to capture the areas around each building and acts as the connection between buildings and streets.
We call this unit morphological tessallation cell and approximates the idea of a plot.

More information is available [here](https://strathprints.strath.ac.uk/70666/1/Fleischmann_etal_CEUS_2019_Morphological_tessellation_as_a_way_of_partitioning_space.pdf) .

In [ ]:
import pandas as pd
import shapely

In [ ]:
idxs = (
    pd.DataFrame(shapely.get_coordinates(buildings.representative_point()))
    .drop_duplicates()
    .index
)

limit = momepy.buffered_limit(buildings.loc[idxs], "adaptive")

tessellation = momepy.morphological_tessellation(buildings, clip=limit)

OpenStreetMap data are often problematic due to low quality of some polygons. If some collapse, we get a mismatch between the length of buildings and the length of polygons.

In [ ]:
collapsed, _ = momepy.verify_tessellation(tessellation, buildings)

In [ ]:
idxs = (
    pd.DataFrame(shapely.get_coordinates(buildings.representative_point()))
    .drop_duplicates()
    .index
)

Better to drop affected buildings and re-create tessellation.

In [ ]:
buildings = buildings.drop(collapsed).reset_index(drop=True)
idxs = (
    pd.DataFrame(shapely.get_coordinates(buildings.representative_point()))
    .drop_duplicates()
    .index
)
limit = momepy.buffered_limit(buildings.iloc[idxs], "adaptive")

tessellation = momepy.morphological_tessellation(buildings, clip=limit)

Check the result.

In [ ]:
tessellation.shape[0] == buildings.shape[0]

### Link streets

Link unique IDs of streets to buildings and tessellation cells based on the nearest neighbor join.

In [ ]:
buildings["street_index"] = momepy.get_nearest_street(
    buildings, streets, max_distance=50
)
buildings

Aattach the network index to the tessellation as well.

In [ ]:
tessellation["street_index"] = buildings["street_index"]

## Measure

Measure individual morphometric characters. For details see the User Guide and the API reference.

### Dimensions

In [ ]:
buildings["building_area"] = buildings.area
tessellation["tess_area"] = tessellation.area
streets["length"] = streets.length

### Shape

In [ ]:
buildings["eri"] = momepy.equivalent_rectangular_index(buildings)
buildings["elongation"] = momepy.elongation(buildings)
tessellation["convexity"] = momepy.convexity(tessellation)
streets["linearity"] = momepy.linearity(streets)

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(24, 12))

# buildings.plot("eri", ax=ax[0], scheme="natural_breaks", legend=True)
# buildings.plot("elongation", ax=ax[1], scheme="natural_breaks", legend=True)

# ax[0].set_axis_off()
# ax[1].set_axis_off()

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(24, 12))

# tessellation.plot("convexity", ax=ax[0], scheme="natural_breaks", legend=True)
# streets.plot("linearity", ax=ax[1], scheme="natural_breaks", legend=True)

# ax[0].set_axis_off()
# ax[1].set_axis_off()

### Spatial distribution

In [ ]:
buildings["shared_walls"] = momepy.shared_walls(buildings) / buildings.length
buildings.plot(
    "shared_walls", figsize=(12, 12), scheme="natural_breaks", legend=True
).set_axis_off()

Generate spatial graph using `libpysal`.

In [ ]:
queen_1 = libpysal.graph.Graph.build_contiguity(tessellation, rook=False)

In [ ]:
queen_1

Due to floating point errors and data quality there are coverage issues with the polygons. Buildings that should have shared walls overlap .001 cm or overlap only at the corners, or have a miniscule distance between them, for example. The same is true for tessellation cells. To account for this we can use fuzzy contiguity when building a graph of spatial relationships.

In [ ]:
queen_1 = libpysal.graph.Graph.build_fuzzy_contiguity(tessellation, buffer=.5)
queen_1

In [ ]:
tessellation["neighbors"] = momepy.neighbors(
    tessellation, queen_1, weighted=True
)
tessellation["covered_area"] = queen_1.describe(tessellation.area)["sum"]
buildings["neighbor_distance"] = momepy.neighbor_distance(buildings, queen_1)

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(24, 12))

# buildings.plot(
#     "neighbor_distance", ax=ax[0], scheme="natural_breaks", legend=True
# )
# tessellation.plot(
#     "covered_area", ax=ax[1], scheme="natural_breaks", legend=True
# )

# ax[0].set_axis_off()
# ax[1].set_axis_off()

In [ ]:
queen_3 = queen_1.higher_order(3)
buildings_q1 = libpysal.graph.Graph.build_contiguity(buildings, rook=False)

buildings["interbuilding_distance"] = momepy.mean_interbuilding_distance(
    buildings, queen_1, queen_3
)
buildings["adjacency"] = momepy.building_adjacency(buildings_q1, queen_3)

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(24, 12))

# buildings.plot(
#     "interbuilding_distance", ax=ax[0], scheme="natural_breaks", legend=True
# )
# buildings.plot("adjacency", ax=ax[1], scheme="natural_breaks", legend=True)

# ax[0].set_axis_off()
# ax[1].set_axis_off()

In [ ]:
profile = momepy.street_profile(streets, buildings)
streets[profile.columns] = profile

In [ ]:
# fig, ax = plt.subplots(1, 3, figsize=(24, 12))

# streets.plot("width", ax=ax[0], scheme="natural_breaks", legend=True)
# streets.plot("width_deviation", ax=ax[1], scheme="natural_breaks", legend=True)
# streets.plot("openness", ax=ax[2], scheme="natural_breaks", legend=True)

# ax[0].set_axis_off()
# ax[1].set_axis_off()
# ax[2].set_axis_off()

### Intensity

In [ ]:
tessellation["car"] = buildings.area / tessellation.area
# tessellation.plot(
#     "car", figsize=(12, 12), vmin=0, vmax=1, legend=True
# ).set_axis_off()

### Connectivity

In [ ]:
graph = momepy.gdf_to_nx(streets)
graph = momepy.node_degree(graph)
graph = momepy.closeness_centrality(graph, radius=400, distance="mm_len")
graph = momepy.meshedness(graph, radius=400, distance="mm_len")
nodes, edges = momepy.nx_to_gdf(graph)

In [ ]:
# fig, ax = plt.subplots(1, 3, figsize=(24, 12))

# nodes.plot(
#     "degree", ax=ax[0], scheme="natural_breaks", legend=True, markersize=1
# )
# nodes.plot(
#     "closeness",
#     ax=ax[1],
#     scheme="natural_breaks",
#     legend=True,
#     markersize=1,
#     legend_kwds={"fmt": "{:.6f}"},
# )
# nodes.plot(
#     "meshedness", ax=ax[2], scheme="natural_breaks", legend=True, markersize=1
# )

# ax[0].set_axis_off()
# ax[1].set_axis_off()
# ax[2].set_axis_off()

In [ ]:
buildings["edge_index"] = momepy.get_nearest_street(buildings, edges)
buildings["node_index"] = momepy.get_nearest_node(
    buildings, nodes, edges, buildings["edge_index"]
)

Link all data together (to tessellation cells or buildings).

In [ ]:
tessellation.head()

In [ ]:
buildings.head()

In [ ]:
tessellation[buildings.columns.drop(["geometry", "street_index"])] = (
    buildings.drop(columns=["geometry", "street_index"])
)
merged = tessellation.merge(
    edges.drop(columns="geometry"),
    left_on="edge_index",
    right_index=True,
    how="left",
)
merged = merged.merge(
    nodes.drop(columns="geometry"),
    left_on="node_index",
    right_index=True,
    how="left",
)

In [ ]:
merged.columns

In [ ]:
attr_columns = merged.columns.drop(
    [
        "street_index",
        "node_index",
        "edge_index",
        "nodeID",
        "mm_len",
        "node_start",
        "node_end",
        "geometry",
        "x",
        "y",
        "id",
        "class",
        "_status",
        # "corine"
        # "shared_walls",
        # "adjacency"
    ]
)

In [ ]:
merged[attr_columns]

### Spatial context

We also calculate statistics to understand the context around each building, by measuring first, second and third quartile of distribution of values within an area around each building. This is one way of using the spatial relationships of the data - using them to calculate features.

In [ ]:
percentiles = []
for column in attr_columns:
    perc = momepy.percentile(merged[column], queen_3)
    perc.columns = [f"{column}_" + str(x) for x in perc.columns]
    percentiles.append(perc)

In [ ]:
percentiles_joined = pd.concat(percentiles, axis=1)
percentiles_joined.head()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(24, 12))

tessellation.plot("convexity", ax=ax[0], scheme="natural_breaks", legend=True)
merged.plot(
    percentiles_joined["convexity_50"].values,
    ax=ax[1],
    scheme="natural_breaks",
    legend=True,
)

ax[0].set_axis_off()
ax[1].set_axis_off()

## Unsupervised machine learning

## Morphotopes

The first step in the analysis is to group our elements into morphotopes using the `SA3` algorithm. Morphotopes are defined as :

> “the smallest urban locality obtaining distinctive character among their neighbours from their particular combination of constituent morphological elements.”

We do this to create contiguous units which can act as the base of the hierarchical analysis. 

This is also an example of the second way of using spatial relationships - directly embeding them in the algorithm.

In [ ]:
from spopt.region import SA3
import numpy
from sklearn.preprocessing import StandardScaler

In [ ]:
attr_columns

In [ ]:
# res = StandardScaler().fit_transform(merged[attr_columns])
# standardised_data = pd.DataFrame(res, columns=attr_columns).fillna(0)

In [ ]:
res = StandardScaler().fit_transform(percentiles_joined)
standardised_data = pd.DataFrame(res, columns=percentiles_joined.columns).fillna(0)

In [ ]:
clusterer = SA3(
    standardised_data,
    queen_1,
    standardised_data.columns,
    min_cluster_size=75,
    extraction="eom",
    linkage='complete'
)
clusterer.solve()
clusterer.labels_.value_counts()

## Clustering

Now we can use the morphotopes to create a hierarchical typology of urban structure. To do this we use simple dendrogram clustering models.


In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

In [ ]:
morph_clusters = clusterer.labels_

# morph_clusters = final_clusters.values

In [ ]:
grouped_data = standardised_data.groupby(morph_clusters).median()
if -1 in grouped_data.index:
    grouped_data = grouped_data.iloc[1:]

In [ ]:
linkage_matrix = linkage(grouped_data, method='complete')

In [ ]:
_ = dendrogram(linkage_matrix)

In [ ]:
morphotope_labels = fcluster(linkage_matrix, t=7.5, criterion='distance')
pd.Series(morphotope_labels).value_counts()

In [ ]:
final_labels = pd.Series(morph_clusters).replace(pd.Series(morphotope_labels, index=grouped_data.index).to_dict())
final_labels.value_counts()

In [ ]:
# assign noise to closest cluster

from sklearn.neighbors import KDTree

cluster_centres = standardised_data.groupby(final_labels).median().iloc[1:]
tree = KDTree(cluster_centres)
dists, idxs = tree.query(standardised_data[final_labels == -1], k=1)
final_labels[final_labels == -1] = idxs[:, 0] + 1

In [ ]:
buildings[["geometry"]].explore(
    column=final_labels, categorical=True, legend=True
)

In [ ]:
merged[attr_columns].groupby(final_labels).median().T.iloc[:, ].style.background_gradient(axis=1)